# Directory

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/amazon-ml/sample_test.csv
/kaggle/input/amazon-ml/sample_test_out_fail.csv
/kaggle/input/amazon-ml/sample_test_out.csv
/kaggle/input/amazon-ml/train.csv
/kaggle/input/amazon-ml/test.csv


# Imports & Others

In [2]:
import os
import random
import warnings
import requests
# import multiprocessing
import re
import time
import urllib
from time import time as timer
from pathlib import Path
from io import BytesIO

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import easyocr
from functools import partial

from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast
from torchvision import transforms

from transformers import BertTokenizer, BertModel


print("Libraries Imported!")

Libraries Imported!


In [3]:
# Suppress specific warnings
warnings.filterwarnings('ignore')

In [4]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 1e-4
MAX_LEN = 128
NUM_WORKERS = 0

# Initialize EasyOCR Reader for text extraction
reader = easyocr.Reader(['en'], gpu=False)

Using device: cuda


In [5]:
entity_unit_map = {
    "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "item_weight": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "maximum_weight_recommendation": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "voltage": {"millivolt", "kilovolt", "volt"},
    "wattage": {"kilowatt", "watt"},
    "item_volume": {"cubic foot", "microlitre", "cup", "fluid ounce", "centilitre", "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart", "cubic inch", "gallon"}
}

all_units = set.union(*entity_unit_map.values())
all_units.add("unknown")  # Add "unknown" as a possible unit

# EDA

In [6]:
train_data = pd.read_csv("/kaggle/input/amazon-ml/train.csv")
train_data.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [8]:
# Regular expression pattern to match: number, space, and unit
pattern = r'^\d+(\.\d+)?\s[a-zA-Z]+$'

# Filter rows that match the pattern
train_data_filtered = train_data[train_data['entity_value'].str.match(pattern)]

In [9]:
train_data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257940 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    257940 non-null  object
 1   group_id      257940 non-null  int64 
 2   entity_name   257940 non-null  object
 3   entity_value  257940 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.8+ MB


In [10]:
# Regular expression pattern to extract the unit part
unit_pattern = r'\s+([a-zA-Z]+)$'

# Extract units from the filtered data
units = train_data_filtered['entity_value'].str.extract(unit_pattern)[0]

# Create a set of unique units
unique_units = set(units.dropna())
unique_units

{'candela',
 'carat',
 'centilitre',
 'centimetre',
 'cup',
 'decilitre',
 'foot',
 'gallon',
 'gigabyte',
 'gram',
 'horsepower',
 'inch',
 'kilogram',
 'kilowatt',
 'litre',
 'metre',
 'microgram',
 'milligram',
 'millilitre',
 'millimetre',
 'nits',
 'ounce',
 'person',
 'pint',
 'pound',
 'quart',
 'ton',
 'volt',
 'watt'}

In [11]:
# Regular expression pattern to extract the unit part
unit_pattern = r'\s+([a-zA-Z]+)$'

# Extract units from the 'entity_value' column
units = train_data_filtered['entity_value'].str.extract(unit_pattern)[0]

# Filter rows where the extracted unit is in the all_units set
train_data_filtered_2 = train_data_filtered[units.isin(all_units)]

# Output the filtered DataFrame
train_data_filtered_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257652 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    257652 non-null  object
 1   group_id      257652 non-null  int64 
 2   entity_name   257652 non-null  object
 3   entity_value  257652 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.8+ MB


In [12]:
# Extract units from the filtered data
units = train_data_filtered_2['entity_value'].str.extract(unit_pattern)[0]

# Create a set of unique units
unique_units = set(units.dropna())
unique_units

{'centilitre',
 'centimetre',
 'cup',
 'decilitre',
 'foot',
 'gallon',
 'gram',
 'inch',
 'kilogram',
 'kilowatt',
 'litre',
 'metre',
 'microgram',
 'milligram',
 'millilitre',
 'millimetre',
 'ounce',
 'pint',
 'pound',
 'quart',
 'ton',
 'volt',
 'watt'}

# Data Loading

In [73]:
def load_data():
    train_df = pd.read_csv('/kaggle/input/amazon-ml/train.csv')
    test_df = pd.read_csv('/kaggle/input/amazon-ml/test.csv')
    
    # Create a validation set
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
    
    return train_df, val_df, test_df

train_df, val_df, test_df = load_data()
train_df = train_data_filtered_2

# Initialize the LabelEncoder for units
unit_encoder = LabelEncoder()
unit_encoder.fit(list(all_units))

# Label encoding for entity_name
label_encoder = LabelEncoder()
label_encoder.fit(train_df['entity_name'])

LabelEncoder()

In [74]:
class ProductImageDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        if not is_test:
            self.df['encoded_entity_name'] = label_encoder.transform(self.df['entity_name'])

    def __len__(self):
        return len(self.df)
    
    def create_placeholder_image():
        return Image.new('RGB', (224, 224), color='black')  

    def download_image(self, url):
        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content)).convert('RGB')
            return img
        except Exception as e:
            print(f"Error downloading image: {e}")
            # Use the placeholder image if there is an error
            return self.create_placeholder_image()

    def __getitem__(self, idx):
        image_url = self.df.iloc[idx]['image_link']
        entity_name = self.df.iloc[idx]['entity_name']
        
        img = self.download_image(image_url)
        
        img = self.transform(img)

        try:
            # OCR Extraction using EasyOCR
            result = reader.readtext(np.array(img.permute(1, 2, 0)), detail=0)
            text = " ".join(result)
        except Exception as e:
            print(f"Error in OCR: {e}")
            text = ""

        # Concatenate entity_name with the OCR extracted text
        combined_text = f"{entity_name} {text}"

        # Tokenize text
        encoding = self.tokenizer.encode_plus(
            combined_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'image': img,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'text': combined_text,
            'entity_name': entity_name
        }

        if not self.is_test:
            item['entity_label'] = torch.tensor(self.df.iloc[idx]['encoded_entity_name'], dtype=torch.long)
            
            # Parse entity value and unit
            entity_value = self.df.iloc[idx]['entity_value']
            value, unit = self.parse_entity_value(entity_value)
            item['entity_value'] = torch.tensor(value, dtype=torch.float)
            
            # Handle empty unit string
            if unit == "":
                unit = "unknown"
            item['entity_unit'] = torch.tensor(unit_encoder.transform([unit])[0], dtype=torch.long)

        return item

    def parse_entity_value(self, entity_value):
        # Handle list-like strings
        if entity_value.startswith('[') and entity_value.endswith(']'):
            parts = entity_value[1:-1].split(',')
        else:
            parts = entity_value.split()
        
        # Extract numeric values
        values = []
        for part in parts:
            try:
                values.append(float(part.strip()))
            except ValueError:
                break
        
        # Calculate average if multiple values
        if values:
            value = sum(values) / len(values)
        else:
            value = 0.0
        
        # Extract unit
        unit = ' '.join(parts[len(values):]).strip()
        
#         print(f"Parsed: value = {value}, unit = {unit}")  # Debug print
        return value, unit

# Models

In [75]:
class EntityExtractionModel(nn.Module):
    def __init__(self, num_labels, num_units):
        super(EntityExtractionModel, self).__init__()
        self.num_labels = num_labels
        self.num_units = num_units
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        self.unit_classifier = nn.Linear(self.bert.config.hidden_size, num_units)

    def forward(self, input_ids, attention_mask, images):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        value = self.regressor(pooled_output).squeeze(-1)
        unit_logits = self.unit_classifier(pooled_output)
        return logits, value, unit_logits

# Training & Inference Functions

In [76]:
def train_model(model, train_loader, val_loader, optimizer, classification_criterion, regression_criterion, unit_criterion, epochs):
    scaler = torch.amp.GradScaler('cuda')
    best_val_loss = float('inf')
    
    for epoch in tqdm(range(epochs), desc='Epoch'):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc="Processing Training Batches"):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            labels = batch['entity_label'].to(device)
            values = batch['entity_value'].to(device)
            units = batch['entity_unit'].to(device)

            with autocast():
                logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
                classification_loss = classification_criterion(logits, labels)
                regression_loss = regression_criterion(predicted_values, values)
                unit_loss = unit_criterion(unit_logits, units)
                loss = classification_loss + regression_loss + unit_loss
                total_loss += loss.item()

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        avg_train_loss = total_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Processing Valid Batches"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                images = batch['image'].to(device)
                labels = batch['entity_label'].to(device)
                values = batch['entity_value'].to(device)
                units = batch['entity_unit'].to(device)

                logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
                classification_loss = classification_criterion(logits, labels)
                regression_loss = regression_criterion(predicted_values, values)
                unit_loss = unit_criterion(unit_logits, units)
                loss = classification_loss + regression_loss + unit_loss
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved best model.")

In [77]:
def inference(model, test_loader, tokenizer):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            entity_names = batch['entity_name']

            logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
            
            # Get predicted entity names
            pred_entity_names = torch.argmax(logits, dim=1)
            pred_entity_names = label_encoder.inverse_transform(pred_entity_names.cpu().numpy())
            
            # Get predicted units
            pred_units = torch.argmax(unit_logits, dim=1)
            pred_units = unit_encoder.inverse_transform(pred_units.cpu().numpy())
            
            # Format predicted values
            for idx, (entity_name, value, unit) in enumerate(zip(entity_names, predicted_values.cpu().numpy(), pred_units)):
                formatted_value = format_value(value, unit, entity_name)
                predictions.append({
                    'index': batch['index'][idx] if 'index' in batch else idx,
                    'prediction': formatted_value
                })

    return pd.DataFrame(predictions)

# Helper Functions

In [ ]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [ ]:
def common_mistake(unit):
    if unit in allowed_units:
        return unit
    if unit.replace('ter', 'tre') in allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in allowed_units:
        return unit.replace('feet', 'foot')
    return unit

In [ ]:
def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, allowed_units))
    return number, unit

In [ ]:
def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

In [ ]:
def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)
    
    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

In [ ]:
def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)

In [78]:
def format_value(value, unit, entity_name):
    if entity_name not in entity_unit_map or unit not in entity_unit_map[entity_name]:
        return ""
    return f"{value:.2f} {unit}".strip()

# Main

*putting it all together*

In [79]:
def main():
    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Create datasets and dataloaders
    train_dataset = ProductImageDataset(train_df, tokenizer, MAX_LEN)
    val_dataset = ProductImageDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = ProductImageDataset(test_df, tokenizer, MAX_LEN, is_test=True)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    # Initialize model
    num_labels = len(label_encoder.classes_)
    num_units = len(set.union(*entity_unit_map.values()))
    model = EntityExtractionModel(num_labels=num_labels, num_units=num_units).to(device)

    # Initialize optimizer and loss functions
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    classification_criterion = nn.CrossEntropyLoss()
    regression_criterion = nn.MSELoss()
    unit_criterion = nn.CrossEntropyLoss()

    # Train the model
    train_model(model, train_loader, val_loader, optimizer, classification_criterion, regression_criterion, unit_criterion, EPOCHS)

    # Load best model for inference
    model.load_state_dict(torch.load('best_model.pth'))

    # Perform inference on test set
    predictions_df = inference(model, test_loader, tokenizer)
    predictions_df.to_csv('output.csv', index=False)
    print("Predictions saved to output.csv")

In [ ]:
main()

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [14]:
import os
import random
import warnings
import requests
import multiprocessing
import re
from io import BytesIO

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from PIL import Image
import easyocr
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms, models

from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

print("Libraries Imported!")

warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 2e-5
MAX_LEN = 128
NUM_WORKERS = multiprocessing.cpu_count()

# Initialize EasyOCR Reader for text extraction
reader = easyocr.Reader(['en'], gpu=True)

entity_unit_map = {
    "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "item_weight": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "maximum_weight_recommendation": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "voltage": {"millivolt", "kilovolt", "volt"},
    "wattage": {"kilowatt", "watt"},
    "item_volume": {"cubic foot", "microlitre", "cup", "fluid ounce", "centilitre", "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart", "cubic inch", "gallon"}
}

all_units = set.union(*entity_unit_map.values())
all_units.add("unknown")

def load_data():
    train_df = pd.read_csv('/kaggle/input/amazon-ml/train.csv')
    test_df = pd.read_csv('/kaggle/input/amazon-ml/test.csv')
    
    train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
    
    return train_df, val_df, test_df

train_df, val_df, test_df = load_data()
train_df = train_data_filtered_2

unit_encoder = LabelEncoder()
unit_encoder.fit(list(all_units))

label_encoder = LabelEncoder()
label_encoder.fit(train_df['entity_name'])

class ProductImageDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        if not is_test:
            self.df['encoded_entity_name'] = label_encoder.transform(self.df['entity_name'])

    def __len__(self):
        return len(self.df)
    
    @staticmethod
    def create_placeholder_image():
        return Image.new('RGB', (224, 224), color='black')  

    def download_image(self, url):
        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content)).convert('RGB')
            return img
        except Exception as e:
            print(f"Error downloading image: {e}")
            return self.create_placeholder_image()

    def __getitem__(self, idx):
        image_url = self.df.iloc[idx]['image_link']
        entity_name = self.df.iloc[idx]['entity_name']
        
        img = self.download_image(image_url)
        img = self.transform(img)

        try:
            result = reader.readtext(np.array(img.permute(1, 2, 0).cpu()), detail=0)
            text = " ".join(result)
        except Exception as e:
            print(f"Error in OCR: {e}")
            text = ""

        combined_text = f"{entity_name} {text}"

        encoding = self.tokenizer.encode_plus(
            combined_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'image': img,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'text': combined_text,
            'entity_name': entity_name
        }

        if not self.is_test:
            item['entity_label'] = torch.tensor(self.df.iloc[idx]['encoded_entity_name'], dtype=torch.long)
            
            entity_value = self.df.iloc[idx]['entity_value']
            value, unit = self.parse_entity_value(entity_value)
            item['entity_value'] = torch.tensor(value, dtype=torch.float)
            
            if unit == "":
                unit = "unknown"
            item['entity_unit'] = torch.tensor(unit_encoder.transform([unit])[0], dtype=torch.long)

        return item

    def parse_entity_value(self, entity_value):
        if entity_value.startswith('[') and entity_value.endswith(']'):
            parts = entity_value[1:-1].split(',')
        else:
            parts = entity_value.split()
        
        values = []
        for part in parts:
            try:
                values.append(float(part.strip()))
            except ValueError:
                break
        
        if values:
            value = sum(values) / len(values)
        else:
            value = 0.0
        
        unit = ' '.join(parts[len(values):]).strip()
        
        return value, unit

class EntityExtractionModel(nn.Module):
    def __init__(self, num_labels, num_units):
        super(EntityExtractionModel, self).__init__()
        self.num_labels = num_labels
        self.num_units = num_units
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.cnn = models.resnet50(pretrained=True)
        self.cnn.fc = nn.Identity()
        
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size + 2048, num_labels)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 2048, 1)
        self.unit_classifier = nn.Linear(self.bert.config.hidden_size + 2048, num_units)

    def forward(self, input_ids, attention_mask, images):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_pooled = bert_output.pooler_output
        
        cnn_features = self.cnn(images)
        
        combined_features = torch.cat((bert_pooled, cnn_features), dim=1)
        combined_features = self.dropout(combined_features)
        
        logits = self.classifier(combined_features)
        value = self.regressor(combined_features).squeeze(-1)
        unit_logits = self.unit_classifier(combined_features)
        
        return logits, value, unit_logits

def train_model(model, train_loader, val_loader, optimizer, scheduler, classification_criterion, regression_criterion, unit_criterion, epochs):
    scaler = GradScaler()
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            labels = batch['entity_label'].to(device)
            values = batch['entity_value'].to(device)
            units = batch['entity_unit'].to(device)

            with autocast():
                logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
                classification_loss = classification_criterion(logits, labels)
                regression_loss = regression_criterion(predicted_values, values)
                unit_loss = unit_criterion(unit_logits, units)
                loss = classification_loss + regression_loss + unit_loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        
        val_loss = validate_model(model, val_loader, classification_criterion, regression_criterion, unit_criterion)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved best model.")

def validate_model(model, val_loader, classification_criterion, regression_criterion, unit_criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            labels = batch['entity_label'].to(device)
            values = batch['entity_value'].to(device)
            units = batch['entity_unit'].to(device)

            with autocast():
                logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
                classification_loss = classification_criterion(logits, labels)
                regression_loss = regression_criterion(predicted_values, values)
                unit_loss = unit_criterion(unit_logits, units)
                loss = classification_loss + regression_loss + unit_loss
            val_loss += loss.item()

    return val_loss / len(val_loader)

def inference(model, test_loader, tokenizer):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Inference"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            entity_names = batch['entity_name']

            with autocast():
                logits, predicted_values, unit_logits = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
            
            pred_entity_names = torch.argmax(logits, dim=1)
            pred_entity_names = label_encoder.inverse_transform(pred_entity_names.cpu().numpy())
            
            pred_units = torch.argmax(unit_logits, dim=1)
            pred_units = unit_encoder.inverse_transform(pred_units.cpu().numpy())
            
            for idx, (entity_name, value, unit) in enumerate(zip(entity_names, predicted_values.cpu().numpy(), pred_units)):
                formatted_value = format_value(value, unit, entity_name)
                predictions.append({
                    'index': idx,
                    'prediction': formatted_value
                })

    return pd.DataFrame(predictions)

def format_value(value, unit, entity_name):
    if entity_name not in entity_unit_map or unit not in entity_unit_map[entity_name]:
        return ""
    return f"{value:.2f} {unit}".strip()

def main():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    train_dataset = ProductImageDataset(train_df, tokenizer, MAX_LEN)
    val_dataset = ProductImageDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = ProductImageDataset(test_df, tokenizer, MAX_LEN, is_test=True)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    num_labels = len(label_encoder.classes_)
    num_units = len(unit_encoder.classes_)
    model = EntityExtractionModel(num_labels=num_labels, num_units=num_units).to(device)

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    
    classification_criterion = nn.CrossEntropyLoss()
    regression_criterion = nn.MSELoss()
    unit_criterion = nn.CrossEntropyLoss()

    train_model(model, train_loader, val_loader, optimizer, scheduler, classification_criterion, regression_criterion, unit_criterion, EPOCHS)

    model.load_state_dict(torch.load('best_model.pth'))

    predictions_df = inference(model, test_loader, tokenizer)
    predictions_df.to_csv('output.csv', index=False)
    print("Predictions saved to output.csv")

if __name__ == '__main__':
    main()

Libraries Imported!
Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error downloading image: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/41uTlpcbCuL.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dd9c58cfbe0>: Failed to resolve 'm.media-amazon.com' ([Errno -3] Temporary failure in name resolution)"))
Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Err

Epoch 1/10:   1%|          | 78/8052 [05:55<67:44:01, 30.58s/it] 

Error downloading image: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/91Tdtz086aL.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dd9c7eb8430>: Failed to resolve 'm.media-amazon.com' ([Errno -3] Temporary failure in name resolution)"))
Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error downloading image: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/81uszczVpGL.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dd9c7828dc0>: Failed to resolve 'm.media-amazon.com' ([Errno -3] Temporary failure in name resolution)"))
Error in OCR: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Error downloading image: HTTPSConnectionPool(host='m.media-amazon.com', port=4

Epoch 1/10:   1%|          | 78/8052 [12:15<20:53:00,  9.43s/it]


KeyboardInterrupt: 